In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.metrics import f1_score
from time import time
from operator import itemgetter
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from __future__ import print_function
import os
import subprocess
from time import time
from operator import itemgetter
from scipy.stats import randint
import pandas as pd
import numpy as np
print(__doc__)
import numpy as np
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from pprint import pprint


filename = 'df_imputed_scaled_OHE_reducedv1.pkl'
df = pd.read_pickle(filename)

X = df[df.columns[:-1]]
y = pd.Series(df['heartdisease'])


train_X,test_X,train_y,test_y =train_test_split(X,y,test_size=0.33,shuffle = True, random_state=45)

/home/javi_fdez_093/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/javi_fdez_093/.local/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Automatically created module for IPython interactive environment


In [2]:
def report(grid_scores, n_top=3):
    """Report top n_top parameters settings, default n_top=3.

    Args
    ----
    grid_scores -- output from grid or random search
    n_top -- how many to report, of top models

    Returns
    -------
    top_params -- [dict] top parameter settings found in
                  search
    """
    top_scores = sorted(grid_scores,
                        key=itemgetter(1),
                        reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print(("Mean validation score: "
               "{0:.3f} (std: {1:.3f})").format(
               score.mean_validation_score,
               np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

    return top_scores[0].parameters

In [3]:
def run_randomsearch(X, y, clf, para_dist, cv=5,
                     n_iter_search=100):
    """Run a random search for best Decision Tree parameters.

    Args
    ----
    X -- features
    y -- targets (classes)
    cf -- scikit-learn Decision Tree
    param_dist -- [dict] list, distributions of parameters
                  to sample
    cv -- fold of cross-validation, default 5
    n_iter_search -- number of random parameter sets to try,
                     default 20.

    Returns
    -------
    top_params -- [dict] from report()
    """
    random_search = RandomizedSearchCV(clf,
                        param_distributions=param_dist,
                        n_iter=n_iter_search, n_jobs=-1)

    start = time()
    random_search.fit(X, y)
    print(("\nRandomizedSearchCV took {:.2f} seconds "
           "for {:d} candidates parameter "
           "settings.").format((time() - start),
                               n_iter_search))

    top_params = report(random_search.grid_scores_, 3)
    return  top_params

In [4]:
print("-- Random Parameter Search via 3-fold CV")

C_range = np.logspace(-5, 5, 11)
gamma_range = np.logspace(-5, 5, 11)
param_dist = {'kernel': ['linear'],
              'gamma': gamma_range,
               'C': C_range}

pprint(param_dist)
SVM_linear = svm.SVC()
SVM_linear_rs = run_randomsearch(X, y, SVM_linear, param_dist, cv=3, n_iter_search=100)

-- Random Parameter Search via 3-fold CV
{'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
 'gamma': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
 'kernel': ['linear']}

RandomizedSearchCV took 2279.95 seconds for 100 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.798 (std: 0.014)
Parameters: {'C': 10000.0, 'kernel': 'linear', 'gamma': 1e-05}

Model with rank: 2
Mean validation score: 0.798 (std: 0.014)
Parameters: {'C': 10000.0, 'kernel': 'linear', 'gamma': 100000.0}

Model with rank: 3
Mean validation score: 0.798 (std: 0.014)
Parameters: {'C': 10000.0, 'kernel': 'linear', 'gamma': 0.001}



In [7]:
# test the retuned best parameters
print("\n\n-- Testing best parameters [Random, CV = 3]...")
SVM_linear_rs_model = svm.SVC(**SVM_linear_rs)
scores = cross_val_score(SVM_linear_rs_model, X, y, cv=10)
print("mean: {:.3f} (std: {:.3f})".format(scores.mean(),
                                          scores.std()),
                                          end="\n\n" )



-- Testing best parameters [Random, CV = 3]...
mean: 0.781 (std: 0.059)



In [8]:
SVM_linear_rs_model = svm.SVC(**SVM_linear_rs)
SVM_linear_base_model = svm.SVC(kernel='linear', C=0.05, gamma = 0.001)



SVM_linear_base_model.fit(train_X, train_y)
SVM_linear_rs_model.fit(train_X, train_y)
prediction_base=SVM_linear_base_model.predict(test_X)
prediction_best=SVM_linear_rs_model.predict(test_X)


print('Test Accuracy for SVM_linear base is ',metrics.accuracy_score(prediction_base,test_y))
print('Test Accuracy for SVM_linear best is ',metrics.accuracy_score(prediction_best,test_y))


prediction_base_train=SVM_linear_base_model.predict(train_X)
prediction_best_train=SVM_linear_rs_model.predict(train_X)
print('Training Accuracy for SVM_linear base is ',metrics.accuracy_score(train_y,prediction_base_train))
print('Training Accuracy for SVM_linear best is ',metrics.accuracy_score(train_y,prediction_best_train))





Test Accuracy for SVM_linear base is  0.8092105263157895
Test Accuracy for SVM_linear best is  0.819078947368421
Training Accuracy for SVM_linear base is  0.7840909090909091
Training Accuracy for SVM_linear best is  0.7873376623376623


In [ ]:
'''
default: ‘liblinear’ Algorithm to use in the optimization problem.
For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and
‘saga’ are faster for large ones.
For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’
handle multinomial loss; ‘liblinear’ is limited to one-versus-rest schemes.
‘newton-cg’, ‘lbfgs’ and ‘sag’ only handle L2 penalty, whereas
‘liblinear’ and ‘saga’ handle L1 penalty.
'''

